# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [217]:
import pandas as pd
import dask.dataframe as dd
import random
import itertools
import numpy as np
from datetime import datetime
from tqdm import tqdm
import nltk
import string
import swifter
from nltk.corpus import stopwords
import fasttext
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importation des données

In [218]:
DBRaw = dd.read_parquet('../data/extraction_sirene_20220510.parquet', engine='pyarrow')

Temporaire : On restreint les données à 1% de la base initiales.

In [219]:
#DBRaw = DBRaw.sample(frac= 0.01, random_state=1234)

On transforme les valeurs manquantes en NaN.

In [220]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [221]:
Var2Keep = ["APE_SICORE","LIB_SICORE","AUTO","DATE","NAT_SICORE","SED_SICORE","EVT_SICORE","SURF"]
DB = DBRaw[Var2Keep]

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [222]:
DB = DB.dropna(subset=['APE_SICORE'])  

On a finalement 10.8 millions de liasses.

In [223]:
DB.shape[0].compute()

10887847

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

### 1) Preprocessing 

In [224]:
DB1 = DB[["APE_SICORE","LIB_SICORE"]]
DB1 = DB1.dropna(subset=['LIB_SICORE'])  

In [225]:
stopwords_ = set(stopwords.words('french') + ['a'])
def CleanLib(lib):
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join([x.lower() for x in lib.split() if x.lower() not in stopwords_])

In [226]:
DB1["LIB_CLEAN"] = DB1["LIB_SICORE"].apply(lambda x: CleanLib(x), meta=pd.Series(dtype='str', name='LIB_CLEAN'))
df = DB1.compute()

### 2) Splitting

On mélange de manière aléatoire les index puis on les divise en 2 groupes selon un certain pourcentage (ici 80% et 20%)


In [ ]:
random.seed(123456)
Idx = random.sample(df.index.values.tolist(), df.shape[0])
Groups = np.split(Idx, [int(len(Idx)*0.8)])

In [ ]:
with open("../data/train_text.txt", 'w') as f:
    for idx in range(len(Groups[0])):
        aLine = "__label__{} {}".format(df.at[Groups[0][idx],"APE_SICORE"], df.at[Groups[0][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)


### 3) Training

On définit plusieurs options pour le modèle.

In [ ]:
config_fasttext={"dim": 150,
"lr": 0.2,
"epoch": 80,
"wordNgrams": 3,
"minn": 3,
"maxn": 4,
"minCount": 3,
"bucket": 3000000,
"thread": 25,
"loss": 'ova',
"label_prefix": '__label__'}

In [ ]:
model1 = fasttext.train_supervised(input="../data/train_text.txt", **config_fasttext)
#model1 = fasttext.load_model("../models/fasttextmodel1.bin")

### Evaluation du modèle

In [ ]:
def get_pred(lib, mod):
    out = mod.predict(lib)
    pred = out[0][0].replace("__label__","")
    prob = out[1][0]
    return [pred, prob]

In [ ]:
df = df.rename(columns={"APE_SICORE":"APE_NIV5"})

Accuracy of the training set

In [ ]:
df_train = df.loc[Groups[0], :]
df_train[['PREDICTION_NIV5', 'PROBA']] = df_train['LIB_CLEAN'].swifter.apply(lambda x: get_pred(x, model1)).to_list()
df_train['GoodPREDICTION'] = df_train['APE_NIV5'] == df_train['PREDICTION_NIV5']
for i in range(2,5):
    df_train['PREDICTION_NIV'+ str(i)] = df_train['PREDICTION_NIV5'].str[:i]
    
sum(df_train['GoodPREDICTION'])/df_train.shape[0] * 100

Accuracy of the testing set

In [ ]:
df_test = df.loc[Groups[1], :]
df_test[['PREDICTION_NIV5', 'PROBA']] = df_test['LIB_CLEAN'].swifter.apply(lambda x: get_pred(x, model1)).to_list()
df_test['GoodPREDICTION'] = df_test['APE_NIV5'] == df_test['PREDICTION_NIV5']
for i in range(2,5):
    df_test['PREDICTION_NIV'+ str(i)] = df_test['PREDICTION_NIV5'].str[:i]

sum(df_test['GoodPREDICTION'])/df_test.shape[0] * 100

On importe un document qui contient des informations sur les différents codes APE

In [ ]:
df_naf =  pd.read_csv(r"../data/naf_extended.csv",sep=",",encoding="utf-8",dtype=str)
df_naf[["NIV3","NIV4","NIV5"]] = df_naf[["NIV3","NIV4","NIV5"]].apply(lambda x: x.str.replace('.', ''))
df_naf.set_index("NIV5", inplace=True, drop=False)

On calcule la prédiction pour le niveau 1 grâce à la table importée

In [ ]:
df_test['PREDICTION_NIV1'] = df_test["PREDICTION_NIV2"].swifter.apply(lambda x : pd.unique(df_naf["NIV1"][df_naf["NIV2"]== x])[0])
df_train['PREDICTION_NIV1'] = df_train["PREDICTION_NIV2"].swifter.apply(lambda x : pd.unique(df_naf["NIV1"][df_naf["NIV2"]== x])[0])

On regarde la precision, le rappel et le F1 pour tous les différentes sous catégories du code APE

In [ ]:
for aNiv in tqdm(["NIV1","NIV2","NIV3","NIV4"]):
    df_test["APE_" + aNiv] = df_test["APE_NIV5"].apply(lambda x : df_naf.loc[x][aNiv])
    df_train["APE_" + aNiv] = df_train["APE_NIV5"].apply(lambda x : df_naf.loc[x][aNiv])

In [ ]:
def get_results(naf, train, test):
    Results = dict()
    for aNiv in ["NIV1","NIV2","NIV3", "NIV4", "NIV5"]:
        Results[aNiv] = train.groupby(['APE_'+ aNiv]).size().rename("Size_TRAIN").to_frame()
        Results[aNiv] = Results[aNiv].join(test.groupby(['APE_'+ aNiv]).size().rename("Size_TEST").to_frame())
        Results[aNiv] = Results[aNiv].join(test.groupby(['APE_'+ aNiv]).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Recall_TEST"}))
        Results[aNiv] = Results[aNiv].join(test.groupby(['PREDICTION_'+ aNiv]).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Precision_TEST"}))
        Results[aNiv] = Results[aNiv].join(2 * 1/(1/Results[aNiv]["Precision_TEST"] + 1/Results[aNiv]["Recall_TEST"]).rename("F1_TEST").to_frame())
        Results[aNiv]["LIB_"+ aNiv] = [pd.unique(naf["LIB_"+ aNiv][naf[aNiv]== x])[0] for x in Results[aNiv].index.values]

    return Results

In [ ]:
Results = get_results(df_naf, df_train, df_test)

In [ ]:
Results["NIV5"]

On calcule les matrices de confusion pour chaque catégories et sous catégorie afin de vérifier la prédiction au sein d'une même catégorie.

In [ ]:
def get_matrix(test, cat, mod):
    sub_cat = "NIV" + str(int(cat[-1]) + 1 )
    filtered_df = df_test[(df_test["APE_" + cat] == mod) & (df_test["PREDICTION_" + cat] == mod)]
    cm = confusion_matrix(filtered_df["APE_" + sub_cat].to_list(), filtered_df["PREDICTION_" + sub_cat].to_list(), normalize = 'true')
    return cm

On regarde d'abord au niveau le plus aggrégé.

In [ ]:
target_names = np.sort(pd.unique(df_test["APE_NIV1"]))
fig, ax = plt.subplots(figsize=(20,8))
sns.heatmap(confusion_matrix(df_test["APE_NIV1"].to_list(), df_test["PREDICTION_NIV1"].to_list(), normalize = 'true'),
             annot=True, fmt='.2f',cmap='Blues', xticklabels=target_names, yticklabels=target_names)

Ensuite on peut regarder les résultats au sein de chaque sous catégorie. Ci-dessous on s'intéresse à la ligne "G" et on vérifie comment sont prédit les codes au sein de cette catégorie.

In [ ]:
ConfMatrices = {aNiv : {mod : get_matrix(df_test, aNiv, mod) for mod in pd.unique(df_test["APE_" + aNiv])} for aNiv in ["NIV1","NIV2","NIV3","NIV4"]}

In [ ]:
aNiv = "NIV1"
aMod = "Q"

aSubNiv = "NIV" + str(int(aNiv[-1]) + 1 )
target_names = np.sort(pd.unique(df_test["APE_" + aSubNiv][df_test["APE_" + aNiv] == aMod]))
fig, ax = plt.subplots(figsize=(20,8))
sns.heatmap(ConfMatrices[aNiv][aMod], annot=True, fmt='.2f',cmap='Blues', xticklabels=target_names, yticklabels=target_names)


- hyper parametres
- checker les libellés, leurs qualité, stats descr 